<h3>Import required dependencies and functions</h3>

In [27]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [70]:
#log in to instagram using username and password
def log_in_to_instagram(username, password):
    driver.find_element_by_xpath("//input[@name='username']").send_keys('username')
    driver.find_element_by_xpath("//input[@name='password']").send_keys('password')
    driver.find_element_by_xpath("//button[contains(.,'Log in')]").click()

#we will need this not to waste resourses while scrapping followers and following links
def define_number_of_links(driver, account, regime=None):
    #if number exceeds million, it should be retrieved from 'title' attribute of a tag
    #else it can be retrieved as .text
    if regime == 'followers':
        soup = BeautifulSoup(driver.page_source, 'lxml')
        try:
            number_of_links = int(soup.find('ul').find_all('li')[1].span['title'].replace(',', ''))
        except KeyError:
            number_of_links = int(soup.find('ul').find_all('li')[1].text.split(' ')[0].replace(',', ''))
        return number_of_links
    elif regime == 'following':
        soup = BeautifulSoup(driver.page_source, 'lxml')
        try:
            number_of_links = int(soup.find('ul').find_all('li')[2].span['title'].replace(',', ''))
        except KeyError:
            number_of_links = int(soup.find('ul').find_all('li')[2].text.split(' ')[0].replace(',', ''))
        return number_of_links
    else:
        print('Please specify regime correctly, options: "followers", "following"')
        return

#Outputs two lists: followers and not_parsed (members of followers list that weren't parsed)
def scrape_followers(driver, account):
    driver.get("https://www.instagram.com/{0}/".format(account))
    #click to open followers list
    driver.find_element_by_partial_link_text("follower").click()
    #element we can apply scrolling to
    scroll_element = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]')
    #make sure list loaded
    time.sleep(3)
    followers = []
    not_parsed = []
    #there we will use number_of_scrolls to save resourses (1 scroll = 12 people)
    for i in range(define_number_of_links(driver, account, 'followers')//12 + 1):
        scroll_element.send_keys(Keys.PAGE_DOWN)
        #browser needs this to load new followers in the list
        time.sleep(0.2)
    #now enjoy classical parsing
    soup = BeautifulSoup(driver.page_source, 'lxml')
    followers_elems = soup.find_all('ul')[-1].find_all('li')
    for fol in followers_elems:
        try:
            name = fol.a['href'].replace('/','')
            followers.append(name)
        except Exception as e:
            print(f'Exception {e} while working with mister {fol}')
            not_parsed.append(fol)
            pass
    return followers, not_parsed

#Outputs two lists: following and not_parsed (members of following links list that weren't parsed)
def scrape_following(driver, account):
    driver.get("https://www.instagram.com/{0}/".format(account))
    #click to open following list
    driver.find_element_by_partial_link_text("following").click()
    #element we can apply scrolling to
    scroll_element = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]')
    #make sure list loaded
    time.sleep(3)
    following = []
    not_parsed = []
    #there we will use number_of_scrolls to save resourses (1 scroll = 12 people)
    for i in range(define_number_of_links(driver, account, 'following')//12 + 1):
        scroll_element.send_keys(Keys.PAGE_DOWN)
        #browser needs this to load new followers in the list
        time.sleep(0.2)
    #now enjoy classical parsing
    soup = BeautifulSoup(driver.page_source, 'lxml')
    following_elems = soup.find_all('ul')[-1].find_all('li')
    for fol in following_elems:
        try:
            name = fol.a['href'].replace('/','')
            following.append(name)
        except Exception as e:
            print(f'Exception {e} while working with mister {fol}')
            not_parsed.append(fol)
            pass
    return following, not_parsed

<h3>Start web driver</h3>

Now we are ready to use our web driver to scrape instagram followers

In [63]:
url = "https://www.instagram.com/accounts/login/"
driver = webdriver.Firefox()
driver.get(url)

<h3>To exploit instagramm at its full potential, log in is required</h3>

In [64]:
log_in_to_instagram('username', 'password')

<h3>Usage</h3>

In [71]:
following, not_parsed = scrape_following(driver, 'mariapoga_')

In [72]:
len(following)

156

In [73]:
following

['serandrew',
 'marinamayer777',
 'i_pracheva',
 'dnk.russia',
 'liubovcharity',
 'mashapoga',
 'sysoevl',
 'schetnikovich',
 'yanasapeta',
 'aliona_hilt',
 'sonya_neks_',
 'glushdaria',
 'karina_maksudova',
 'pimenov.rus',
 'alinabure',
 'boshavoroshilova',
 'nikitaefremov',
 'satikazanova',
 'nata_p1502',
 'yulia.enhel',
 'mirosmeister',
 'borisovna_77',
 'fc_ural',
 'veraangel',
 'zhdunovna',
 'akif.tamince',
 'stasanbro',
 'mesutsoltay',
 'arina_2003',
 'darianorkina',
 'iamzlatam',
 'itsbatchaeva',
 '_rina_bombina_',
 'evgeniyakukova',
 'margarita_vn',
 'gontveronika',
 'marina_ross_atelier',
 'masha_gridnevskaya',
 'redgafur',
 'butterfly_xtina',
 'kruglikova_director',
 'kisse__a',
 'tariq_royal',
 'irina_official.rus',
 'veronika_krav',
 'tatiana_v_lyalina',
 'ekaterinamiga4eva',
 'buslya',
 'dmitry_bess2017',
 'voguerussia',
 'stepanov.i.dm',
 'kalachikm',
 'cheryshev90',
 'grebenkina_official',
 'oh_werner',
 'yagnetinskaya',
 'zaharov._ilya',
 'katyagordon',
 'elenabushina',

<h3>Shut down web driver</h3>

Don't forget to close the web driver

In [74]:
driver.quit()